In [ ]:
!nvidia-smi

Fri Mar 19 08:37:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pickle
import spacy
spacy.require_gpu()
import random
from spacy.util import minibatch, compounding
from spacy import load, displacy
import random
from tqdm import tqdm
#nlp=spacy.blank("id")
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
input_dir = '/content/drive/My Drive/ner_shopee_daerah'
nlp = spacy.load(input_dir)
#nlp.add_pipe(nlp.create_pipe('ner'))
ner = nlp.get_pipe('ner')
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def cekOverlap(src_start, src_end, posisi):
  idx = 0
  if(posisi >= src_start and posisi <= src_end):
    idx = src_end
    return idx
  return idx
  
df = pd.read_csv('/content/drive/My Drive/train.csv')
import re
print(len(df))
df_baru = df[df['POI/street'] != '/']
print(len(df_baru))
df_baru_copy = df_baru.copy()
raw = {}
true = {}
ctr = 0
for data in df_baru_copy.values:
  split_true_result = data[2].split('/')
  cek_typo_poi = 0
  cek_typo_loc = 0
  cek_mandarin = 0
  if(re.search(u'[\u4e00-\u9fff]', data[1])): #hapus mandarin
    cek_mandarin = 1
  if(split_true_result[0] not in data[1]):  #hapus yang typo
    cek_typo_poi = 1
  if(split_true_result[1] not in data[1]):
    cek_typo_loc = 1
  if(cek_typo_poi == 0 and cek_typo_loc == 0 and cek_mandarin == 0 and '+' not in data[1]):
    s_poi = data[1].find(split_true_result[0])
    e_poi = (s_poi+len(split_true_result[0]))

    if(split_true_result[0] != '' and split_true_result[1] != ''):
      s_loc = data[1].find(split_true_result[1])
      res = cekOverlap(s_poi, e_poi, s_loc)
      if(res != 0):
        s_loc = data[1].find(split_true_result[1], res)
        if(s_loc != -1):
          raw[ctr] = {data[1].lower()}
          true[ctr] = {data[2].lower()}
          ctr += 1
        else:
          print(data[1])
      else:
        raw[ctr] = {data[1].lower()}
        true[ctr] = {data[2].lower()}
        ctr += 1
    else:
      raw[ctr] = {data[1].lower()}
      true[ctr] = {data[2].lower()}
      ctr += 1
print(len(raw))

300000
268007
brilink duadelima del sentang
jl. lewi anyar gg. bungsu rt. 06 rw. 03 kelas sukamanah kec ci pedes kota tasikmalaya
jaga, no 104 bakso jagalan 87, rw 16 genteng
pasirhonje, pasirh 40399 cimeunyan
jl. leuwianyar gg. bungsu no 56 rt.03 rw.03 kel. sukamanah
karang dalem soto bebek rajawali raja, 14
jl. lewianyar gg. bungsu rt. 06 rw. 03 kel. sukamanah kec cipedes kota tasikmalaya
pps sukamulya garawangi, garaw no 82
tukad pancoran 4, tukad panc, panjer denpasar selatan
bahagia bekasi bek 36
idm ft3m osamaliki, osama
warkop arimbi, ari, palmerah
bengkel panaitan, pana 3
jl. leuwianyar gg. bungsu rt. 06 rw. 03 kel. sukamanah kec. cipedes kota tasikmalaya
jln mojopahit tuban, mojop, sido rejo
kost pria galunggung, gal, lengkong
kebon kaw, no 22e damri kantor pemasaran kebon kawung 20, cicendo
maguwoharjo denokan deno, 52 depok
soto ayam ambengan sadi asli, ambe,
perum leman mand, perum leman mandiri
richeese factory adityawarman, adityaw wonokromo
jl. lewi anyar gg. bungsu rt. 

In [ ]:
daerah = {}
df_kota = pd.read_csv('/content/drive/My Drive/daerah.csv')
print(len(df_kota))
cek_kembar = 0
for data in df_kota.values:
  if(data[0].lower() in daerah):
    cek_kembar += 1
  daerah[data[0].lower()] = data[0].lower()
print("redundant = "+str(cek_kembar))
print(len(daerah))

88261
redundant = 22954
65307


In [ ]:
start_poi = {}
end_poi = {}
start_street = {}
end_street = {}
start_daerah = {}
end_daerah = {}
start_informasi = {}
end_informasi = {}
df_baru.head(10)

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru
5,5,"raya samb gede, 299 toko bb kids",toko bb kids/raya samb gede
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra...",/kem mel raya
7,7,tela keuramat kuta alam,/tela
8,8,gg. i wates magersari,/gg. i
9,9,bunga ncole ix 2,/bunga ncole ix
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj


In [ ]:
def checkConsecutive(l): 
    n = len(l) - 1
    return (sum(np.diff(sorted(l)) == 1) >= n)  

def isntValueInBetween(index, start, end, tipe = 0):
  return_value = 0
  #print(str(index))
  #print('posisi = '+str(posisi_yang_mau_dicek))
  if(index in start_poi):
    #print('masuk if poi')
    #print(str(list(start_poi[index])[0]))
    #print(str(list(end_poi[index])[0]))
    if(start >= list(start_poi[index])[0] and start <= list(end_poi[index])[0] and list(end_poi[index])[0] != 0):
      #print(str(list(start_poi[index])[0])+' inbetween 1')
      #print(posisi_yang_mau_dicek)
      #print(str(list(end_poi[index])[0])+' inbetween 1')
      return_value = list(end_poi[index])[0]
      return return_value
    elif(end >= list(start_poi[index])[0] and end <= list(end_poi[index])[0] and list(end_poi[index])[0] != 0):
      #print(str(list(start_poi[index])[0])+' inbetween 1')
      #print(posisi_yang_mau_dicek)
      #print(str(list(end_poi[index])[0])+' inbetween 1')
      return_value = list(end_poi[index])[0]
      return return_value
    elif(end >= list(end_poi[index])[0] and start <= list(start_poi[index])[0] and list(end_poi[index])[0] != 0):
      return_value = list(end_poi[index])[0]
      return return_value
  if(index in start_street):
    if(start >= list(start_street[index])[0] and start <= list(end_street[index])[0] and list(end_street[index])[0] != 0):
      #print(str(list(start_street[index])[0])+' inbetween 2')
      #print(posisi_yang_mau_dicek)
      #print(str(list(end_street[index])[0])+' inbetween 2')
      return_value = list(end_street[index])[0]
      return return_value
    elif(end >= list(start_street[index])[0] and end <= list(end_street[index])[0] and list(end_street[index])[0] != 0):
      #print(str(list(start_street[index])[0])+' inbetween 2')
      #print(posisi_yang_mau_dicek)
      #print(str(list(end_street[index])[0])+' inbetween 2')
      return_value = list(end_street[index])[0]
      return return_value
    elif(end >= list(end_street[index])[0] and start <= list(start_street[index])[0] and list(end_street[index])[0] != 0):
      return_value = list(end_street[index])[0]
      return return_value
  if(index in start_daerah and tipe == 0): #tipe untuk pengecekan akhir pembentukan label
    list_end = list(sorted(end_daerah[index]))
    list_start = list(sorted(start_daerah[index]))
    for i in range(len(list_end)):
      if(start >= list_start[i] and start <= list_end[i] and list_end[i] != 0):
        return list_end[i]
      elif(end >= list_start[i] and end <= list_end[i] and list_end[i] != 0):
        return list_end[i]
  return return_value

def getInformationIndex(index):
  result = []
  ctr = 0
  print(raw[index])
  #print('1')
  #print(result)
  
  if(index in end_poi):
    if(0 not in end_poi[index]):
      end = 0
      start = 0
      for val in end_poi[index]:
        end = int(val)
      for val in start_poi[index]:
        start = int(val)
      for i in range(start,end+1,1):
        result.append(i)
  #print('2')
  #print(result)
  #print(end_street[index])
  if(index in end_street):
    if(0 not in end_street[index]):
      end = 0
      start = 0
      for val in end_street[index]:
        end = int(val)
      for val in start_street[index]:
        start = int(val)
      for i in range(start,end+1,1):
        result.append(i)

  #print('3')
  #print(result)
  if(0 not in end_daerah[index]):
    end = []
    start = []
    for val in end_daerah[index]:
      end.append(val)
    for val in start_daerah[index]:
      start.append(val)
    for i in range(len(start)):
      for j in range(start[i], end[i]+1, 1):
        result.append(j)
  #print('4')
  #print(result)
  arr_informasi_temp = []
  temp_final = []
  final_start = []
  final_end = []
  clone_raw = (''.join(raw[index]))
  cek_digit_di_start = 0
  for i in range(len(clone_raw)+1):
    if(i not in result):
      arr_informasi_temp.append(i)
      if(checkConsecutive(arr_informasi_temp) == False):
        #print(final_start)
        if(clone_raw[arr_informasi_temp[0]] == ' '):
          #print('masuk if spasi')
          if(i < len(clone_raw)):
            if(clone_raw[i] != ',' and clone_raw[i] != ' ' and clone_raw[i] != '!'
            and clone_raw[i] != ')' and clone_raw[i] != ')'):
              final_start.append(arr_informasi_temp[1]) #buang spasinya
          else:
            if(clone_raw[-1] != ',' and clone_raw[-1] != ' ' and clone_raw[-1] != '!'
            and clone_raw[-1] != ')' and clone_raw[-1] != ')'):
              text = clone_raw[-1]
              pattern=r'[^A-Za-z\s]'
              text=re.sub(pattern,'',text)
              if(text == '' or text == []):
                cek_digit_di_start = 1
              final_start.append(arr_informasi_temp[1]) #buang spasinya
        else:
          #print('else : '+str(arr_informasi_temp[0]))
          #print('else :' +str(clone_raw[arr_informasi_temp[0]]))
          final_start.append(arr_informasi_temp[0])
        #print(i)
        if(i >= len(clone_raw) and clone_raw[-1] != ',' and clone_raw[-1] != ' ' and clone_raw[-1] != '!'
        and clone_raw[-1] != ')' and clone_raw[-1] != '.' and clone_raw[-1] != '-' and clone_raw[-1] != "'"): #jika char berada di ujung akhir raw
          #print('masuk akhir')
          cek_digit = clone_raw[-1]
          #print('digit = '+cek_digit)
          pattern=r'[^A-Za-z\s]'
          cek_digit=re.sub(pattern,'',cek_digit)
          #print('digit 2= '+cek_digit)
          if((cek_digit != [] and cek_digit != '') or cek_digit_di_start == 1):
            #print('masuk gan')
            final_end.append(arr_informasi_temp[-1])
          else:
            final_end.append(arr_informasi_temp[-2])
        else:
          #print('masuk else -2')
          final_end.append(arr_informasi_temp[-2])
        arr_informasi_temp = [i]

  #print('final start = '+str(final_start))
  #print('final end = '+str(final_end))

  if(len(arr_informasi_temp) > 1): #tambah data terakhir yang gk masuk fungsi append
    if(clone_raw[arr_informasi_temp[0]] == ' '):
      final_start.append(arr_informasi_temp[1]) #buang spasinya
    else:
      final_start.append(arr_informasi_temp[0])
    
    final_end.append(arr_informasi_temp[-1])
  #print('not in : '+str(arr_informasi_temp))
  
  #print(arr_informasi_temp)
  #print('final start = '+str(final_start))
  #print('final end = '+str(final_end))

  final_start = sorted(final_start)
  final_end = sorted(final_end)
  o_start = 0
  o_end = 0
  idx_counter = 0
  
  for val in final_start:
    cek_overlap = isntValueInBetween(index, val, (val+final_end[idx_counter]))
    if(cek_overlap != 0):
      del final_start[idx_counter]
      del final_end[idx_counter]
    idx_counter += 1

  return sorted(final_start), sorted(final_end)

daerah.pop('utara', None)
daerah.pop('barat', None)
daerah.pop('selatan', None)
daerah.pop('timur', None)
daerah.pop('tengah', None)

'tengah'

In [ ]:
start_poi = {}
end_poi = {}
start_street = {}
end_street = {}
start_daerah = {}
end_daerah = {}
start_informasi = {}
end_informasi = {}
import numpy as np

for i in range(len(raw)):
  string_raw = (''.join(raw[i]))
  split_true = (''.join(true[i])).split('/')
  idx_POI = string_raw.find(split_true[0])
  if((idx_POI == 0 or idx_POI == -1) and split_true[0] == ''): print('tdk ditemukan poi')
  
  new_raw = string_raw[:idx_POI] + string_raw[idx_POI+len(split_true[0]):]
  idx_street = string_raw.find(split_true[1])

  if(idx_street < idx_POI):
    if(idx_street != -1):
      start_street[i] = {idx_street}
      idx_end = (idx_street+len(split_true[1]))
      end_street[i] = {idx_end}
      idx_new_POI = string_raw.find(split_true[0], idx_end)
      if(idx_new_POI != -1):
        start_poi[i] = {idx_new_POI}
        end_poi[i] = {idx_new_POI+len(split_true[0])}
      else:
        start_poi[i] = {0}
        end_poi[i] = {0}
    else:
      start_poi[i] = {idx_POI}
      end_poi[i] = {idx_POI+len(split_true[0])}
  else:
    if(idx_POI != -1):
      start_poi[i] = {idx_POI}
      idx_end = idx_POI+len(split_true[0])
      end_poi[i] = {idx_end}
      idx_new_street = string_raw.find(split_true[1], idx_end)
      if(idx_new_street != -1):
        start_street[i] = {idx_new_street}
        end_street[i] = {(idx_new_street+len(split_true[1]))}
      else:
        start_street[i] = {0}
        end_street[i] = {0}
    else:
      start_street[i] = {idx_street}
      end_street[i] = {(idx_street+len(split_true[1]))}


  new_idx_street = new_raw.find(split_true[1])
  if((idx_street == 0 or idx_street == -1) and split_true[1] == ''): print('tdk ditemukan street')
  #dibuat new_raw untuk memisahkan nama daerah dari nama tempat, contoh poi: sd negeri bojong, nama daerah bojong
  #namun ini kurang efektif, kalau .find, pasti akan ketemu bojong milik sd negeri, solusi pakai fungsi inbetween
  new_raw = new_raw[:new_idx_street] + new_raw[new_idx_street+len(split_true[1]):]

  split_cari_daerah = new_raw.split(' ')
  str_daerah_combined = ''
  ada_daerah = 0
  past_daerah_combined = ''
  start_daerah[i] = [] #nanti dihapus nilai 100000
  end_daerah[i] = []
  ctr_daerah = 0
  arr_word = []
  for word in split_cari_daerah:
    
    word = word.replace(',', '').replace('.', '').replace('(','').replace(')','')
    word = word.strip()
    if(word != '' or word != ' '):
      start_daerah[i] = sorted(start_daerah[i])
      end_daerah[i] = sorted(end_daerah[i])
      if(word.lower() in daerah):

        #print('word '+word )
        past_daerah_combined = str_daerah_combined
        str_daerah_combined += ' '+word
        #print('word '+str_daerah_combined )
        ada_daerah = 1
      else:
        if(str_daerah_combined != ''):
          #print('NYARI APA LAGI '+word)
          
          ada_daerah = 2
          str_daerah_combined = str_daerah_combined.strip()
          temp_simpan = (''.join(str_daerah_combined))
          #print('INI COMBINE = '+str_daerah_combined)
          #print('INI PAST = '+past_daerah_combined)
          #print(str(start_daerah[i]))
          #print(str(end_daerah[i]))
          if(str_daerah_combined.lower() not in daerah): #perlu buat cicau cikarang pusat, beda nya ini pake NOT IN, gk digabung sama if dibawah karena untuk str_daerah_combined +word
            #print(str_daerah_combined)
            past_daerah_combined = past_daerah_combined.strip()
            split_past = past_daerah_combined.split(' ')
            #print(past_daerah_combined+' fffffff '+word+' ffffff '+str_daerah_combined)

            idx_start = 0
            #cek apakah kata ini pernah ada sebelumnya, kayak surabaya timur rt 1 rw 3, surabaya, 
            #kalau gk dicek, nanti bakal ngambil posisi surabaya timur, bkn surabaya terakhir
            for idx_word in range(len(arr_word)): 
              if((past_daerah_combined in arr_word[idx_word]) or (arr_word[idx_word] in past_daerah_combined)):
                ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                for val in end_daerah[i]:
                  if(ctr == idx_word):
                    idx_start = val
                    #print('idx start 0 = '+str(idx_start))
                  ctr += 1

            idx_daerah = string_raw.find(past_daerah_combined, idx_start)
            #print(str(idx_daerah) + ' - '+str(idx_start) )
            return_value = isntValueInBetween(i, idx_daerah, idx_daerah+len(past_daerah_combined))
            #print('return = '+str(return_value))
            if(return_value != 0 and return_value != -1):
              idx_daerah = string_raw.find(past_daerah_combined, return_value)
              #print(str(idx_daerah)+' baru')
            if(idx_daerah != -1):
              #print('masuk 0')
              return_value2 = isntValueInBetween(i, idx_daerah, idx_daerah+len(past_daerah_combined))
              if(return_value2 != 0 and return_value2 != -1):
                idx_daerah = string_raw.find(past_daerah_combined, return_value2)
              #print(string_raw)
              if(idx_daerah != -1):
                arr_word.append(past_daerah_combined)
                start_daerah[i].append(idx_daerah)
                end_daerah[i].append(idx_daerah+len(past_daerah_combined))
              str_daerah_combined = str_daerah_combined.replace((past_daerah_combined), '')
              #print(str_daerah_combined)
            else: #asumsi ada 4 kata di loop ini, 2 kata pertama adalah daerah yang berbeda2 sehingga gk bisa nemu dgn cara if diatas ini
              kata1 = split_past[0]
              idx_start = 0
              #print(split_past)
              for idx_word in range(len(arr_word)): 
                if((kata1 in arr_word[idx_word]) or (arr_word[idx_word] in kata1)):
                  ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                  for val in end_daerah[i]:
                    if(ctr == idx_word):
                      idx_start = val
                      #print('idx start 00 = '+str(idx_start))
                    ctr += 1
              search1 = string_raw.find(kata1, idx_start)
              endSearch1 = search1+len(kata1)
              return_value = isntValueInBetween(i, search1, endSearch1)

              if(return_value != 0 and return_value != -1):
                search1 = string_raw.find(kata1, return_value)
              if(search1 != -1):
                #print('masuk 00')
                return_value2 = isntValueInBetween(i, search1, endSearch1)
                if(return_value2 != 0 and return_value2 != -1):
                  search1 = string_raw.find(kata1, return_value2)
                #print(string_raw)
                if(search1 != -1):
                  arr_word.append(kata1)
                  start_daerah[i].append(search1)
                  end_daerah[i].append(search1+len(kata1))
                str_daerah_combined = str_daerah_combined.replace((kata1), '')
              if(len(split_past) == 2):
                kata2 = split_past[1]
                idx_start = 0
                for idx_word in range(len(arr_word)): 
                  if((kata2 in arr_word[idx_word]) or (arr_word[idx_word] in kata2)):
                    ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                    for val in end_daerah[i]:
                      if(ctr == idx_word):
                        idx_start = val
                        #print('idx start 01 = '+str(idx_start))
                      ctr += 1
                search2 = string_raw.find(kata2, idx_start)
                return_value = isntValueInBetween(i, search2, (search2+len(kata2)))

                if(return_value != 0 and return_value != -1):
                  search2 = string_raw.find(kata2, return_value)

                if(search2 != -1):
                  #print('masuk 01')
                  return_value2 = isntValueInBetween(i, search2, (search2+len(kata2)))
                  if(return_value2 != 0 and return_value2 != -1):
                    search2 = string_raw.find(kata2, return_value2)
                  #print(string_raw)
                  if(search2 != -1):
                    arr_word.append(kata2)
                    start_daerah[i].append(search2)
                    end_daerah[i].append(search2+len(kata2))
                  str_daerah_combined = str_daerah_combined.replace((kata2), '')

          #print('strip remove '+str_daerah_combined)
          str_daerah_combined += ' '+word #word yang gk ditambah, karena gk masuk if pertama di loop ini
          str_daerah_combined = str_daerah_combined.strip()
          #print('DIGABUNG JADI = '+str_daerah_combined)
          str_daerah_combined_future = ''
          if(ctr_daerah+1 >= len(split_cari_daerah)): #cek 3 kata, kalau uda mentok di ujung, gk ditambah kata baru
            str_daerah_combined_future = str_daerah_combined.strip()
          else: #tambah kata selanjutnya, karena kalau gk ditambah sekarang, loop selanjutnya gk kedeteksi daerah 3 kata
            str_daerah_combined_future = (str_daerah_combined + ' '+ split_cari_daerah[ctr_daerah+1]).strip()
          #print('temp simpan : '+temp_simpan)
          if(temp_simpan in daerah): #kata sebelum word saat ini
            #cek jika word sekarang, nyambung sama word sebelumnya
            #print('DIGABUNG JADI 2 = '+str_daerah_combined)
            #print('temp = '+temp_simpan)
            if(str_daerah_combined in daerah): #jika daerah punya 2 kata
              idx_start = 0
              #cek apakah kata ini pernah ada sebelumnya, kayak surabaya timur rt 1 rw 3, surabaya, 
              #kalau gk dicek, nanti bakal ngambil posisi surabaya timur, bkn surabaya terakhir
              for idx_word in range(len(arr_word)): 
                if((str_daerah_combined in arr_word[idx_word]) or (arr_word[idx_word] in str_daerah_combined)):
                  ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                  for val in end_daerah[i]:
                    if(ctr == idx_word):
                      #print('idx start 1 = '+str(idx_start))
                      idx_start = val
                    ctr += 1
              #print('masuk 1')
              idx_daerah = string_raw.find(str_daerah_combined, idx_start) #kayak bojong rawalumbu, 2 kata terpisah masing2 juga nama daerah
              return_value = isntValueInBetween(i, idx_daerah, (idx_daerah+len(str_daerah_combined)))
              if(return_value != 0 and return_value != -1):
                idx_daerah = string_raw.find(str_daerah_combined, return_value)

              if(idx_daerah != -1):
                return_value2 = isntValueInBetween(i, idx_daerah, (idx_daerah+len(str_daerah_combined)))
                if(return_value2 != 0 and return_value2 != -1):
                  idx_daerah = string_raw.find(str_daerah_combined, return_value2)
                #print(string_raw)
                if(idx_daerah != -1):
                  arr_word.append(str_daerah_combined)
                  start_daerah[i].append(idx_daerah)
                  end_daerah[i].append(idx_daerah+len(str_daerah_combined))
            else: #daerah 1 kata, find kata sebelum 'word' di loop ini
              idx_start = 0
              #cek apakah kata ini pernah ada sebelumnya, kayak surabaya timur rt 1 rw 3, surabaya, 
              #kalau gk dicek, nanti bakal ngambil posisi surabaya timur, bkn surabaya terakhir
              for idx_word in range(len(arr_word)): 
                #print('for : '+arr_word[idx_word]+', '+temp_simpan)
                if((temp_simpan in arr_word[idx_word]) or (arr_word[idx_word] in temp_simpan)):
                  #print('masuk if for nest')
                  ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                  for val in end_daerah[i]:
                    if(ctr == idx_word):
                      idx_start = val
                      #print('masuk for nest '+str(idx_start))
                    ctr += 1
              #print('masuk 2')
              #print(temp_simpan+' , idx start 2 = '+str(idx_start))
              idx_daerah = string_raw.find(temp_simpan, idx_start)
              #print('dddd '+str(start_daerah[i]))
              #print('dddd '+str(end_daerah[i]))
              #print('dddd '+str(idx_daerah))
              return_value = isntValueInBetween(i, idx_daerah, idx_daerah+len(temp_simpan))
              if(return_value != 0 and return_value != -1):
                idx_daerah = string_raw.find(temp_simpan, return_value)
                #print('eee '+str(idx_daerah))

              if(idx_daerah != -1):
                return_value2 = isntValueInBetween(i, idx_daerah, idx_daerah+len(temp_simpan))
                if(return_value2 != 0 and return_value2 != -1):
                  idx_daerah = string_raw.find(temp_simpan, return_value2)
                #print(string_raw)
                if(idx_daerah != -1):
                  arr_word.append(temp_simpan)
                  start_daerah[i].append(idx_daerah)
                  end_daerah[i].append(idx_daerah+len(temp_simpan))
          elif(str_daerah_combined in daerah):
            if(str_daerah_combined_future in daerah):
              idx_start = 0
              #cek apakah kata ini pernah ada sebelumnya, kayak surabaya timur rt 1 rw 3, surabaya, 
              #kalau gk dicek, nanti bakal ngambil posisi surabaya timur, bkn surabaya terakhir
              for idx_word in range(len(arr_word)): 
                if((str_daerah_combined_future in arr_word[idx_word]) or (arr_word[idx_word] in str_daerah_combined_future)):
                  ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                  for val in end_daerah[i]:
                    if(ctr == idx_word):
                      idx_start = val
                    ctr += 1
              #print('masuk 3')
              idx_daerah = string_raw.find(str_daerah_combined_future, idx_start)
              return_value = isntValueInBetween(i, idx_daerah, idx_daerah+len(str_daerah_combined_future))
              if(return_value != 0 and return_value != -1):
                idx_daerah = string_raw.find(str_daerah_combined_future, return_value)

              if(idx_daerah != -1):
                return_value2 = isntValueInBetween(i, idx_daerah, idx_daerah+len(str_daerah_combined_future))
                if(return_value2 != 0 and return_value2 != -1):
                  idx_daerah = string_raw.find(str_daerah_combined_future, return_value2)
                #print(string_raw)
                if(idx_daerah != -1):
                  arr_word.append(str_daerah_combined_future)
                  start_daerah[i].append(idx_daerah)
                  end_daerah[i].append(idx_daerah+len(str_daerah_combined_future))
            elif(str_daerah_combined in daerah):
              idx_start = 0
              #cek apakah kata ini pernah ada sebelumnya, kayak surabaya timur rt 1 rw 3, surabaya, 
              #kalau gk dicek, nanti bakal ngambil posisi surabaya timur, bkn surabaya terakhir
              for idx_word in range(len(arr_word)): 
                #print('for 4:'+str(str_daerah_combined)+', '+arr_word[idx_word])
                if((str_daerah_combined in arr_word[idx_word]) or (arr_word[idx_word] in str_daerah_combined)):
                  ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
                  for val in end_daerah[i]:
                    if(ctr == idx_word):
                      idx_start = val
                      #print('idx start 4 ='+str(idx_start) )
                    ctr += 1
              #print('masuk 4')
              idx_daerah = string_raw.find(str_daerah_combined, idx_start)
              return_value = isntValueInBetween(i, idx_daerah, idx_daerah+len(str_daerah_combined))
              if(return_value != 0 and return_value != -1):
                idx_daerah = string_raw.find(str_daerah_combined, return_value)

              if(idx_daerah != -1):
                return_value2 = isntValueInBetween(i, idx_daerah, idx_daerah+len(str_daerah_combined))
                if(return_value2 != 0 and return_value2 != -1):
                  idx_daerah = string_raw.find(str_daerah_combined, return_value2)
                #print(string_raw)
                if(idx_daerah != -1):
                  #print(str(idx_daerah)+' duhhh')
                  arr_word.append(str_daerah_combined)
                  start_daerah[i].append(idx_daerah)
                  end_daerah[i].append(idx_daerah+len(str_daerah_combined))

          print(start_daerah[i])
          print(end_daerah[i])
          #print(past_daerah_combined)
          #print(str_daerah_combined_future)
          #print(str_daerah_combined)
            
        str_daerah_combined = ''
    ctr_daerah += 1

  print('ada daerah '+str(int(ada_daerah)) )
  #print(end_daerah[i])
  start_daerah[i] = sorted(start_daerah[i])
  end_daerah[i] = sorted(end_daerah[i])
  if(ada_daerah == 1):
    #print('abc '+str_daerah_combined.strip())
    str_daerah_combined = str_daerah_combined.strip()
    idx_start = 0
    #cek apakah kata ini pernah ada sebelumnya, kayak surabaya timur rt 1 rw 3, surabaya, 
    #kalau gk dicek, nanti bakal ngambil posisi surabaya timur, bkn surabaya terakhir

    for idx_word in range(len(arr_word)): 
      #print('daerah 1 '+arr_word[idx_word])
      if((str_daerah_combined in arr_word[idx_word]) or (arr_word[idx_word] in str_daerah_combined)):
        #print('masuk daerah 1')
        ctr = 0 #duh gk suka set, gk isa lgsg ambil nilai index ke x
        for val in end_daerah[i]:
          #print(val)
          if(ctr == idx_word):
            idx_start = val
          ctr += 1
    idx_daerah = string_raw.find(str_daerah_combined, idx_start)
    return_value = isntValueInBetween(i, idx_daerah, (idx_daerah+len(str_daerah_combined)))
    if(return_value != 0 and return_value != -1):
      idx_daerah = string_raw.find(str_daerah_combined, return_value)

    if(idx_daerah != -1):
      return_value2 = isntValueInBetween(i, idx_daerah, (idx_daerah++len(str_daerah_combined)))
      if(return_value2 != 0 and return_value2 != -1):
        idx_daerah = string_raw.find(str_daerah_combined, return_value2)
      #print(string_raw)
      if(idx_daerah != -1):
        start_daerah[i].append(idx_daerah)
        end_daerah[i].append(idx_daerah+len(str_daerah_combined))

  start_daerah[i] = sorted(start_daerah[i])
  end_daerah[i] = sorted(end_daerah[i])
  if(ada_daerah == 0):
    start_daerah[i] = {0}
    end_daerah[i] = {0}
  
  
  print(str(start_daerah[i]))
  print(str(end_daerah[i]))
  
  start_informasi[i], end_informasi[i] = getInformationIndex(i)
  #print(start_informasi[i], end_informasi[i])
  #print('=============================================')
  #print('=============================================')

Streaming output truncated to the last 5000 lines.
ada daerah 1
[17, 53]
[25, 61]
{'perumahan wahana cikarang blk c7 no 11 rt 008 rw 009 sukadami'}
tdk ditemukan poi
ada daerah 0
{0}
{0}
{'bin, no 52b 20211'}
tdk ditemukan poi
ada daerah 1
[12]
[27]
{'komp pet 33 tanah baru beji'}
[18, 28]
[27, 40]
ada daerah 2
[18, 28]
[27, 40]
{'kantor pos, perm, citeureup cimahi utara'}
tdk ditemukan poi
[20]
[29]
ada daerah 1
[20, 32]
[29, 40]
{'gay kebon xii no 15 ketintang 5 gayungan'}
ada daerah 0
{0}
{0}
{'ricko ban, jend. sudi,'}
tdk ditemukan poi
[0]
[17]
ada daerah 1
[0, 42]
[17, 51]
{'kembangan selatan atla i, 23 rt 2 1 11610 kembangan'}
tdk ditemukan poi
ada daerah 1
[14]
[22]
{'menja 5 30961 kalidoni'}
tdk ditemukan street
ada daerah 0
{0}
{0}
{'apartemen gunawangsa manyar menur pumpungan .titip securitt unit 618 tower b'}
ada daerah 0
{0}
{0}
{'show room 3r motor citeureup'}
tdk ditemukan poi
ada daerah 1
[4]
[20]
{'kel pondok cina beji'}
tdk ditemukan poi
ada daerah 1
[11]
[31]
{'profe 

In [ ]:
train = []

for i in range(len(raw)):
  start_poi[i] = list(set(start_poi[i]))
  end_poi[i] = list(set(end_poi[i]))
  start_daerah[i] = list(set(start_daerah[i]))
  end_daerah[i] = list(set(end_daerah[i]))
  start_informasi[i] = list(set(start_informasi[i]))
  end_informasi[i] = list(set(end_informasi[i]))
  start_street[i] = list(set(start_street[i]))
  end_street[i] = list(set(end_street[i]))

  a = {}
  b = []
  clone_raw = (''.join(raw[i]))
  b.append(clone_raw) #raw txt
  a['entities'] = []

  start_poi[i] = sorted(start_poi[i])
  end_poi[i] = sorted(end_poi[i])
  start_street[i] = sorted(start_street[i])
  end_street[i] = sorted(end_street[i])
  if(list(end_poi[i])[0] > 0 and list(start_poi[i])[0] >=0):
    a['entities'].append([int(start_poi[i][0]), int(end_poi[i][0]), 'TEMPAT'])
  if(list(end_street[i])[0] > 0 and list(start_street[i])[0] >=0):
    a['entities'].append([int(start_street[i][0]), int(end_street[i][0]), 'LOKASI'])

  arr_start = sorted(start_daerah[i])
  arr_end = sorted(end_daerah[i])
  if(0 in start_daerah[i] and 9 in end_daerah[i] and list(start_street[i])[0] == 3 and list(end_street[i])[0] == 4):
    print(i)
    print(raw[i])

  for j in range(len(start_daerah[i])):
    cek_overlap = isntValueInBetween(i, arr_start[j], arr_end[j], 1)
    if(arr_end[j] > 0 and cek_overlap == 0):
      a['entities'].append([arr_start[j], arr_end[j], 'DAERAH'])
  
  b.append(a)
  train.append(b)
print(train[:100])
#113530

[['jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat', {'entities': [[0, 40, 'LOKASI'], [46, 51, 'DAERAH'], [52, 66, 'DAERAH']]}], ['setu siung 119 rt 5 1 13880 cipayung', {'entities': [[5, 10, 'LOKASI'], [0, 4, 'DAERAH'], [28, 36, 'DAERAH']]}], ['toko dita, kertosono', {'entities': [[0, 9, 'TEMPAT'], [9, 9, 'LOKASI'], [11, 20, 'DAERAH']]}], ['jl. orde baru', {'entities': [[0, 13, 'LOKASI']]}], ['raya samb gede, 299 toko bb kids', {'entities': [[20, 32, 'TEMPAT'], [0, 14, 'LOKASI']]}], ['kem mel raya, no 4 bojong rawalumbu rt 1 36 rawalumbu', {'entities': [[0, 12, 'LOKASI'], [19, 35, 'DAERAH'], [44, 53, 'DAERAH']]}], ['tela keuramat kuta alam', {'entities': [[0, 4, 'LOKASI'], [5, 23, 'DAERAH']]}], ['gg. i wates magersari', {'entities': [[0, 5, 'LOKASI'], [6, 21, 'DAERAH']]}], ['bunga ncole ix 2', {'entities': [[0, 14, 'LOKASI']]}], ['abim ix 24 5', {'entities': [[0, 7, 'LOKASI']]}], ['gang xiii rungkut', {'entities': [[0, 9, 'LOKASI'], [10, 17, 'DAERAH']]}], ['kampung.

In [ ]:
print(train[:100])

{'jl. orde baru'}
[0]


In [ ]:
for _, annotations in train:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [ ]:
optimizer = nlp.begin_training()

with nlp.disable_pipes(*unaffected_pipes):
  for iteration in range(50):

    # shuufling examples  before every iteration
    random.shuffle(train)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train, size=compounding(1.0, 128.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    sgd=optimizer,
                    drop=0.3,  # dropout - make it harder to memorise data
                    losses=losses,
                )
    
    print("Losses at iteration {}".format(iteration), losses)

Losses at iteration 0 {'ner': 84951.19861456138}
Losses at iteration 1 {'ner': 82847.43979414261}
Losses at iteration 2 {'ner': 82480.70710230962}
Losses at iteration 3 {'ner': 81534.2380420826}
Losses at iteration 4 {'ner': 80786.09646388196}
Losses at iteration 5 {'ner': 81045.0944494346}
Losses at iteration 6 {'ner': 80948.1179228856}
Losses at iteration 7 {'ner': 80676.65744142515}
Losses at iteration 8 {'ner': 80398.18211555801}
Losses at iteration 9 {'ner': 80341.69742546816}
Losses at iteration 10 {'ner': 80143.41224698728}
Losses at iteration 11 {'ner': 80392.73732835957}
Losses at iteration 12 {'ner': 79689.52307497656}
Losses at iteration 13 {'ner': 79546.81787436473}
Losses at iteration 14 {'ner': 79652.50490726948}
Losses at iteration 15 {'ner': 78794.41066243243}


In [ ]:
output_dir = '/content/drive/My Drive/ner_shopee_daerah2'

nlp.to_disk(output_dir)
print("Saved model to", output_dir)

pred_col = []
df_test = pd.read_csv('/content/drive/My Drive/test.csv')
for i, row in df_test.iterrows():
  result = nlp(row['raw_address'])
  tempat = list([ent.text for ent in result.ents if(ent.label_ == 'TEMPAT')])
  if(len(tempat) != 0):
    tempat = ''.join(tempat)
  else:
    tempat = ''
  lokasi = list([ent.text for ent in result.ents if(ent.label_ == 'LOKASI')])
  if(len(lokasi) != 0):
    lokasi = ''.join(lokasi)
  else:
    lokasi = ''
  pred_col.append(tempat + "/" + lokasi)

In [ ]:

df_test['POI/street'] = pred_col
df_test.to_csv('/content/drive/My Drive/submission3label2.csv', index = False)